In [4]:
import msoffcrypto
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import locale
from utils import calculate_age, drop_columns, average_fill_empty, calculate_malnutrition, drop_cap_nutrition_rows, generate_gender_column_and_carelevel, check_missing_values, fill_missing_by_idno_and_mode
from dotenv import load_dotenv
import os

# Reading an encrypted Excel file
encrypted_file_path = "./datasets/InterRAI Pt 4 2505.xlsx"
target_folder = "./datasets/"
# Use password in .env
load_dotenv()
# Reading an encrypted Excel file
# encrypted_file_path = os.getenv("CSV_PATH")
password = os.getenv("PASSWORD")


with open(encrypted_file_path, "rb") as f:
    office_file = msoffcrypto.OfficeFile(f)
    office_file.load_key(password=password)
    # Decrypt the file
    decrypted = io.BytesIO()
    office_file.decrypt(decrypted)

    dfs = pd.read_excel(decrypted, sheet_name=None, engine="openpyxl")
basic_info_df = dfs["Staying Upright InterRAI"]
extra_info_df = dfs["Staying Upright demo2"]

In [5]:
# Preprocess the DataFrames
df = calculate_age(basic_info_df, extra_info_df)
df = generate_gender_column_and_carelevel(basic_info_df, extra_info_df)
# # Check for missing values
# columns_to_check = ['iJ1g', 'iJ1h', 'iJ1i', 'iJ12', 'iK1ab', 'iK1bb']
# check_missing_values(columns_to_check, df)
df = drop_columns(df)

    IDno        Age
0  01001  87.166667
1  01001  87.750000
2  01001  87.916667
3  01001  88.083333
4  01001  88.416667
5  01001  88.916667
6  01001  89.416667
7  01002  84.250000
8  01002  84.666667
9  01002  85.166667
   CareLevel  Gender
0        1.0     2.0
1        3.0     2.0
2        3.0     1.0
3        3.0     1.0
4        3.0     1.0
5        1.0     2.0
6        3.0     1.0
7        1.0     2.0
8        1.0     2.0
9        3.0     2.0


In [6]:
# Generate cap df
cap_df = drop_cap_nutrition_rows(df)
cap_df = fill_missing_by_idno_and_mode(cap_df)
cap_df = calculate_malnutrition(cap_df)
cap_df.to_pickle(target_folder + 'cap_data.pkl')
# df = pd.read_pickle(target_folder + 'cap_data.pkl')

In [7]:
# Generate mal df
mal_df = fill_missing_by_idno_and_mode(df)
mal_df = calculate_malnutrition(df)
mal_df = mal_df.drop(columns=["iK2a", "iK2g", "iG3", "iE2a", "iE2b", "iE2c", "iI1c", "iI1d", "CAP_Nutrition"])
mal_df.to_pickle(target_folder + 'mal_data.pkl')
# mal_df = pd.read_pickle(target_folder + 'mal_data.pkl')